# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
****

In [17]:
import pandas as pd
import numpy as np
from scipy import stats
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [5]:
# number of callbacks for white-sounding names
sum(data[data.race=='w'].call)

235.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [8]:
len(data.call)
data.race.value_counts()

w    2435
b    2435
Name: race, dtype: int64

In [22]:
b_call = data[data.race=='b'].call
w_call = data[data.race=='w'].call

**Question 1:** What test is appropriate for this problem? Does CLT apply?

<ul>Becuase we are comparing two groups, and the sample population is greater than 30, a two-sample $t$-test is the appropriate choice. Our data set has an equal number of resumes from both groups. CLT does apply.

**Question 2:** What are the null and alternate hypotheses?

<ul>**Null Hypothesis:** The rate of callbacks (mean of column 'call') for white sounding names and black sounding names  are the same.

**Alternative Hypothesis:** The rate of callbacks for each type of name are different.

**Question 3:** Compute margin of error, confidence interval, and p-value.

In [45]:
#observed mean difference
diff_mean = w_call.mean()-b_call.mean()
print("Difference in mean callback rate:", round(diff_mean, 4))
perc_dmean = ((w_call.mean() - b_call.mean())/w_call.mean())*100
print(perc_dmean)


#standard error: t-test, two samples
std_err = np.sqrt(b_call.var()/b_call.size + w_call.var()/w_call.size)
print("Standard Error:", std_err.round(4))

Difference in mean callback rate: 0.032
33.19148992017526
Standard Error: 0.0078


In [38]:
#degrees of freedom: t-test, two samples
var_b = b_call.var()
var_w = w_call.var()

n_b = len(b_call)
n_w = len(w_call)

DF = int((var_b/n_b + var_w/n_w)**2 / (((var_b/n_b)**2 / (n_b-1)) + ((var_w/n_w)**2 / (n_w-1))))
print("Degrees of freedom:", DF)

Degrees of freedom: 4711


In [41]:
#critical value: t-test, two samples
t_crit = stats.t.ppf(0.975,df=DF)

#margin of error:
margin = t_crit * std_err
print("Margin of error:",margin)

Margin of error: 0.0152620581617


In [40]:
#95% confidence interval: t-test, two samples
CI_95 = [diff_mean - margin, diff_mean + margin]
print("95% Confidence interval:",CI_95)

95% Confidence interval: [0.016770796693321634, 0.047294913016799521]


In [42]:
#t-test
stats.ttest_ind(w_call, b_call, equal_var=False)

Ttest_indResult(statistic=4.1147052908617514, pvalue=3.9429415136459352e-05)

**Question 4:** Write a story describing the statistical significance in the context or the original problem.

<ul>Our low $p$-value (<0.05) indicates we should reject the null hypothesis, and accept the hypothesis that states the call back rate for white sounding names and black sound names are distinctly different. In fact, based on these data, white sounding names receive call backs at a *33% greater rate*.

**Question 5:** Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

<ul> This study was set up to randomly assign the type of name to each resume.  However, race (and or the sound of the name) is not necessarily the most important factor in callback rate.  One would need to develop a multivariate analysis to evaluate the impact of ALL the factors related to call back success (assuming information about other factors is available).